## WEM project: JVC Review Bombing Detection

#### Project's members:  Campos Carvalho Cédric, Feuillade Florian, Ramosaj Nicolas

#### Import Libraires

In [1]:
import pandas as pd
import numpy as np
from review_bombing import *
from tools.read import get_data

%load_ext autoreload
%autoreload 2

#### Load Dataset

In [2]:
dataset = get_data('data/dataset500.csv')
data = [{'game':dataset['name'][i], 'platform':dataset['platform'][i], 
        'grade':comment['grade'][0],'comment':comment['comment'][0],'username':comment['username'][0]}
        for i, comments in enumerate(dataset['comments']) for comment in comments]
df = pd.DataFrame(data, columns=['game', 'platform', 'comment', 'username', 'grade'])
df.head()
#print(len(df))

,game,platform,comment,username,grade
0,Elden Ring,PlayStation 5,"Comme Socrate disait:""les personnes qui font s...",Platon_Socrate,20
1,Elden Ring,PlayStation 5,La quintessence de la formule des Souls.Miyaza...,Chichariito,19
2,Elden Ring,PlayStation 5,Le jeu est excellent et tourne bien sur PS5 en...,TheRealBigBoss,20
3,Elden Ring,PlayStation 5,"Je suis là pour corriger ma note, j'ai retiré ...",keanu77fr,18
4,Elden Ring,PlayStation 5,Tout à fait novice dans les jeux Fromsoftware ...,enguerrand_92,19


#### Data normalization

In [3]:
df['comment_normalized'] = [normalize_lemm_stem(comment) for comment in df['comment']]

In [4]:
df.head()

,game,platform,comment,username,grade,comment_normalized
0,Elden Ring,PlayStation 5,"Comme Socrate disait:""les personnes qui font s...",Platon_Socrate,20,comme socrate disaitles personnes font subir p...
1,Elden Ring,PlayStation 5,La quintessence de la formule des Souls.Miyaza...,Chichariito,19,quintessence formule soulsmiyazaki continue co...
2,Elden Ring,PlayStation 5,Le jeu est excellent et tourne bien sur PS5 en...,TheRealBigBoss,20,jeu excellent tourne bien ps5 mode performance...
3,Elden Ring,PlayStation 5,"Je suis là pour corriger ma note, j'ai retiré ...",keanu77fr,18,corriger note jai retiré précédent commentaire...
4,Elden Ring,PlayStation 5,Tout à fait novice dans les jeux Fromsoftware ...,enguerrand_92,19,tout fait novice jeux fromsoftware jai jouer q...


#### Statistical Analysis

In [5]:
df_grade_pos = df[df['grade'] > 18]
print("Number of comments with grade 19 or 20:", len(df_grade_pos))
df_grade_neg = df[df['grade'] < 2]
print("Number of comments with grade 0 or 1:", len(df_grade_neg))

Number of comments with grade 19 or 20: 2705
Number of comments with grade 0 or 1: 1168


In [6]:
names_pos = extreme_behaviour(dataframe=df, sentiment='positive')
names_neg = extreme_behaviour(dataframe=df, sentiment='negative')
print("Number of user with a positive behaviour:", len(names_pos))
print("Number of user with a negative behaviour:", len(names_neg))

Number of user with a positive behaviour: 246
Number of user with a negative behaviour: 249


#### Prepare Review Bombing Words and Extract Malicious Comments

In [7]:
print(get_bombing_words())

['0', '0/20', '20', '20/20', 'augmente', 'augmenté', 'augmenter', 'augmentes', 'augmentés', 'baisse', 'baissé', 'baisser', 'baisses', 'baissés', 'bombing', 'boycott', 'boycotts', 'bug', 'bugs', 'colere', 'coleres', 'colère', 'colères', 'contre', 'contré', 'contrer', 'contres', 'contrés', 'descendre', 'descendu', 'descendus', 'ensemble', 'ensembles', 'equilibre', 'equilibrer', 'equilibres', 'equilibré', 'equilibrés', 'équilibre', 'équilibrer', 'équilibres', 'équilibré', 'équilibrés', 'evaluation', 'évaluation', 'evalue', 'evalué', 'évalue', 'évalué', 'evaluer', 'évaluer', 'evalues', 'evalués', 'évalues', 'évalués', 'gars', 'honte', 'honteuse', 'honteuses', 'honteux', 'jeuxvideo', 'jeuxvideocom', 'jvc', 'koi', 'mauvais', 'mauvaise', 'mauvaises', 'mec', 'mecs', 'negatif', 'negatifs', 'negative', 'negatives', 'négatif', 'négatifs', 'négative', 'négatives', 'note', 'noter', 'notes', 'patch', 'patche', 'patcher', 'patché', 'positif', 'positifs', 'positive', 'positives', 'quoi', 'rage', 'ragé

In [8]:
review_bombing_pos = naive_bombing(dataframe=df, sentiment='positive')
review_bombing_neg = naive_bombing(dataframe=df, sentiment='negative')
print(len(review_bombing_pos), len(review_bombing_neg))

1251 712


In [9]:
review_bombing_pos = review_bombing_pos[review_bombing_pos['username'].isin(names_pos)]
review_bombing_neg = review_bombing_neg[review_bombing_neg['username'].isin(names_neg)]

#### Positive sentiment analysis Vader

In [10]:
# review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game='Elden Ring')
review_bombing_pos = extract_game_sentiment(dataframe=review_bombing_pos, game='Gran Turismo 7')

C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['compound_vader'] = [senti['compound'] for senti in senti_vader]
C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['polarity_tb'] = [senti[0] for senti in sentiment_blob]
C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [11]:
print(len(review_bombing_pos))
review_bombing_pos.to_csv('pos.csv', encoding='utf-8-sig')

8


In [12]:
pred = predict_review_bombing(dataframe=review_bombing_pos, sentiment='positive')
print("Comment:", pred[0], "From", pred[1])

Comment: 13 sur 20 par la communauté, pourtant on à ici un jeux sérieux solide et qui respecte les passionnés de voitures.
Ok gt7 n'est plus le port- étendard de la simulation de voitures par excellence, comme il l'était a une époque.
C'est pas une révolution graphique ou même niveaux sensation.
Mais honnêtement le jeux fait plus que le taff, un 16 ou 17 c'est pas volé mais voilla comme certains sous côte car le jeux n'est pas pensé comme eux l'on pensé ou par mauvaise foies simple ou se retrouve avec des exclus sony basher. Last of us, horizon, gt7 franchement c'est triste de chier sur un travail de qualité. Jusqu'à quand seigneur, jusqu'à quand. From Kounic89


In [16]:
pred = predict_review_bombing(dataframe=review_bombing_pos, sentiment=None)
print("Comment:", pred[0], "From", pred[1])

Comment: J'ai le jeu depuis la sortie.
Aucunement embeté par la connexion obligatoire..
On aurrais pu s'en passer, mais qui n'as pas internet en 2022. Un faux problème.
Les mictotransactions trop présentent ? Pas plus que dans cod et fifa, et ne pas sortir la cb ne vous handicaperas pas.
60h de jeux, je n'ai pas finis les 'menus' (quêtes en solo), et j'ai 2 permis sur 5.. Une page de defis sur 6.
On dirais qu'on parle d'un jeu avec 3 voitures et 2 circuits, et le reste en dlc payant... C'est pas le cas.Ces 0, ce review bombing massif me dégoûte de la communauté des 'joueurs', qui, en 2022, ne se fait pas un avis en jouant, mais en imitant tel des lemmings les dires de leurs youtubeurs ou streamers préfférés, adeptent des dramas et titres putacliquiens pour le buzz.
On dirais qu'on parle d'un cyberpunk injouable à sa sortie, alors que là, on as un jeu finis et bien finis, sans bugs, et qui peut vous tennir plus de 50h juste avec son contenu d'origine.
Bien plus en refesant des courses d

#### Negative sentiment analysis with TextBlob and Vader

In [13]:
# review_bombing_neg = extract_game_sentiment(dataframe=review_bombing_neg, game='Elden Ring')
review_bombing_neg = extract_game_sentiment(dataframe=review_bombing_neg, game='Gran Turismo 7')

C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['compound_vader'] = [senti['compound'] for senti in senti_vader]
C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['polarity_tb'] = [senti[0] for senti in sentiment_blob]
C:\Users\Nicolas\switchdrive\MA_WEM\projetWEM\review_bombing.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [14]:
print(len(review_bombing_neg))
review_bombing_neg.to_csv('neg.csv', encoding='utf-8-sig')

11


In [15]:
pred = predict_review_bombing(dataframe=review_bombing_neg, sentiment='negative')
print("Comment:", pred[0], "From", pred[1])

Comment: Comment est-ce possible ? Un jeu qui devait tout avoir pour plaire !Un côté RPG à l'ancienne allié à la capacité de la console dernière génération !On sent l'amour de la voiture des développeurs à chaque minute, quel dommage de gâcher ça avec l'amour de l'argent, probablement ajouté de force par les développeurs à cause d'une hiérarchie définitivement peu au courant de comment marche le monde du jeu vidéo.Un jeu vendu 80€, bourré de micro-transactions aux prix plus que prohibitifs (qui va dépenser 120€ dans une voiture sur un jeu qui coûte déjà aussi cher ??!). Et quand les joueurs râlent puis trouvent des alternatives, on les sanctionne en baissant les récompenses !Les récompenses parlons-en, chaque jour on gagne un "ticket de loterie". Ne comptez pas trop sur votre bonne étoile, les dés sont pipés. regardez-bien le plus petit lot que vous pourrez avoir avec ledit ticket: c'est celui que vous aurez 95% du temps. Cette statistique n'est pas une extrapolation. Graphiquement nou